In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import casadi
import numpy as np
import pinocchio

from ncsudyn.models.loader import get_robot_model
from ncsudyn.visualize.visualizer import visualize_trajectory

In [ ]:
# Second cell - system setup and dynamics
# System parameters
m = 0.775  # mass (kg)
L = 0.15  # arm length (m)
I = casadi.diag([0.0015, 0.0025, 0.0035])  # moment of inertia
kF = 1.0  # force constant
kM = 0.0245  # moment constant
g = 9.81  # gravity (m/s^2)

# Define optimization variables
T = 5.0  # Time horizon
N = 100  # Number of control intervals
dt = T / N

# Define state and control variables
nx = 12  # [x, y, z, roll, pitch, yaw, vx, vy, vz, wx, wy, wz]
nu = 4  # [F1, F2, F3, F4]

# Create symbolic variables
x = casadi.SX.sym("x", nx)
u = casadi.SX.sym("u", nu)

# Extract states
pos = x[:3]
rpy = x[3:6]
vel = x[6:9]
omega = x[9:]


# Rotation matrix function
def rotation_matrix(rpy):
    roll, pitch, yaw = rpy[0], rpy[1], rpy[2]
    R_x = casadi.vertcat(
        casadi.horzcat(1, 0, 0),
        casadi.horzcat(0, casadi.cos(roll), -casadi.sin(roll)),
        casadi.horzcat(0, casadi.sin(roll), casadi.cos(roll)),
    )
    R_y = casadi.vertcat(
        casadi.horzcat(casadi.cos(pitch), 0, casadi.sin(pitch)),
        casadi.horzcat(0, 1, 0),
        casadi.horzcat(-casadi.sin(pitch), 0, casadi.cos(pitch)),
    )
    R_z = casadi.vertcat(
        casadi.horzcat(casadi.cos(yaw), -casadi.sin(yaw), 0),
        casadi.horzcat(casadi.sin(yaw), casadi.cos(yaw), 0),
        casadi.horzcat(0, 0, 1),
    )
    return R_z @ R_y @ R_x


# Calculate dynamics
R = rotation_matrix(rpy)
F = casadi.sum1(u)
Fgravity = casadi.vertcat(0, 0, -m * g)
Faero = R @ casadi.vertcat(0, 0, F)

M = casadi.vertcat(L * (u[1] - u[3]), L * (u[2] - u[0]), kM * (u[0] - u[1] + u[2] - u[3]))

acc = (Fgravity + Faero) / m
alpha = casadi.solve(I, M - casadi.cross(omega, I @ omega))

x_dot = casadi.vertcat(vel, omega, acc, alpha)

# Create dynamics function
dynamics_fn = casadi.Function("dynamics_fn", [x, u], [x_dot])


# Euler integrator
def euler_integrate(x, u):
    x_next = x + dynamics_fn(x, u) * dt
    return x_next


opti = casadi.Opti()

# Create optimization variables
X = opti.variable(nx, N + 1)
U = opti.variable(nu, N)

# Initial and final conditions
x0 = np.zeros(12)  # start at origin
xf = np.array([-5, -5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])  # end at (5,5,5)

# Add waypoints
N_third = N // 3
N_two_thirds = 2 * N // 3
waypoint1 = np.array([-3, 3, -3])  # First waypoint
waypoint2 = np.array([2, 4, 3])  # Second waypoint

opti.subject_to(X[:, 0] == x0)
opti.subject_to(X[:, -1] == xf)

# Add waypoint constraints (only for position)
opti.subject_to(X[:3, N_third] == waypoint1)
opti.subject_to(X[:3, N_two_thirds] == waypoint2)

# Dynamic constraints
for k in range(N):
    x_next = euler_integrate(X[:, k], U[:, k])
    opti.subject_to(X[:, k + 1] == x_next)

# Control constraints
hover_thrust = m * g / 4
opti.subject_to(opti.bounded(0, U, 2 * hover_thrust))

# Modify objective function to encourage smooth transitions
obj = 0
for k in range(N):
    obj += casadi.sumsqr(U[:, k])  # Control effort
    if k < N - 1:
        obj += 1e-1 * casadi.sumsqr(U[:, k + 1] - U[:, k])  # Smooth control
        obj += 1e2 * casadi.sumsqr(X[:, k + 1] - X[:, k])  # Smooth state transitions

opti.minimize(obj)

# Solve
p_opts = {"expand": True}
s_opts = {"max_iter": 1000}
opti.solver("ipopt", p_opts, s_opts)
sol = opti.solve()

# Extract solution
X_opt = sol.value(X)
U_opt = sol.value(U)

In [4]:
from ncsudyn.core.optimize import Trajectory

time_trajectory = list()
for k in range(N + 1):
    time_trajectory.append(k * dt)


Q_trajectory = list()
for k in range(N + 1):
    x = X_opt[:, k]
    x_new = [0 for _ in range(6)]
    x_new[:3] = x[:3]
    x_new[3:] = x[3:6][::-1]
    Q_trajectory.append(x_new)

traj = Trajectory(np.array(Q_trajectory).T, np.array([]), np.array([]), np.array(time_trajectory))
# traj.to_file("quadrotor_trajectory.traj")

In [5]:
joint_composite = pinocchio.JointModelComposite()
joint_composite.addJoint(pinocchio.JointModelPX())
joint_composite.addJoint(pinocchio.JointModelPY())
joint_composite.addJoint(pinocchio.JointModelPZ())
joint_composite.addJoint(pinocchio.JointModelRZ())
joint_composite.addJoint(pinocchio.JointModelRY())
joint_composite.addJoint(pinocchio.JointModelRX())

m = get_robot_model("quadrotor", root_joint=joint_composite)

In [ ]:
from pinocchio.visualize import MeshcatVisualizer

m.setVisualizer(MeshcatVisualizer())
m.initViewer(open=True)
m.loadViewerModel()

In [7]:
import time

from ncsudyn.core.interpolate import LinearInterpolator

dt = 0.01
tt = 0

while tt < traj.time_traj[-1]:
    q = LinearInterpolator.interpolate(tt, traj.time_traj, traj.Q_traj.transpose())
    m.display(q)
    tt += dt
    time.sleep(dt)

In [8]:
visualize_trajectory(m, traj, add_past=True, base_link="x2")